In [1]:
import gc
import re

import numpy as np

"""
ToyNetwork 재분석
    램프 : 진입
    집계시간 : 5분
    분석시간 1800~8400
    검지기 : 841개
"""
import pandas as pd

import os

# FIX 값 모음
###################################################################################################################

path = r"C:\VISSIM_Workspace\테스트\램프0_진입램프\mer 통합"

start_interval = 1800
end_interval = 8400

weights = {
    "w1" : 1,
    "w2" : 1,
    "w3" : 1,
    "w4" : 1,
    "w5" : 1,
    "w6" : 1
}

vehicle_types = [100, 300, 630, 640, 650]

enter_line = [1]

# 램프 전 본선 검지기
before_ramp = [49]

# 램프 후 본선 검지기
after_ramp = [53]

# 유입 검지기
input_ramp = [999]

# 유출 검지기
output_ramp = [0]

###################################################################################################################

# 함수 모음
###################################################################################################################

# 속도 변화율
def speed_mean(original_df):
    copy_df = original_df.copy()

    # 램프 검지기 제외
    copy_df = copy_df[~copy_df["New_Measurement"].between(900, 910)]

    # TimeGroup, New_Measurement별 그룹화 및 속도 평균
    speed_mean_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement"])
          .agg(V_mean=("v[km/h]", "mean"), V_count=("v[km/h]", "count"))
          .reset_index()
    )
    speed_mean_df["V_next"] = speed_mean_df.groupby("TimeGroup")["V_mean"].shift(-1)
    speed_mean_df["delta_V"] = (speed_mean_df["V_next"] - speed_mean_df["V_mean"]) / speed_mean_df["V_mean"]
    speed_mean_df["delta_V"] = speed_mean_df["delta_V"].fillna(0)

    return speed_mean_df

# 밀도 변화율
def density_mean(speed_df):
    copy_df = speed_df.copy()
    density_mean_df = copy_df.assign(K = copy_df["V_count"] * 12 / copy_df["V_mean"])
    density_mean_df["K_next"] = density_mean_df.groupby("TimeGroup")["K"].shift(-1)
    density_mean_df["delta_K"] = (density_mean_df["K_next"] - density_mean_df["K"]) / density_mean_df["K"]
    density_mean_df["delta_K"] = density_mean_df["delta_K"].fillna(0)
    return density_mean_df

# 중차량 혼입률
def heavy_rate(original_df):
    copy_df = original_df.copy()

    heavy_df = (
        copy_df[copy_df["Vehicle type"].isin([630, 640, 650])]
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="heavy_count")
    )
    # TimeGroup별 총 차량 갯수 집계
    total_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="total_count")
    )
    heavy_rate_df = pd.merge(
        heavy_df,
        total_df,
        on=["TimeGroup", "New_Measurement"],
        how="left"
    )
    heavy_rate_df["rate"] = heavy_rate_df["heavy_count"] / heavy_rate_df["total_count"]
    return heavy_rate_df


# 동적 포화도
def entry_saturation(original_df):
    copy_df = original_df.copy()

    # 실측용량 C(3차로 6600)
    max_capacity = 6600
    entry_saturation_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="entry_volume")  # 차량 수를 entry_volume이라는 컬럼명으로
    )

    # 단위가 대/시 이기 때문에 현재 5분집계 * 12
    entry_saturation_df["Phi_진입"] = entry_saturation_df["entry_volume"] * 12 / max_capacity
    return entry_saturation_df

# 램프 간섭 영향률
def rfr_rate(original_df):
    copy_df = original_df.copy()
    copy_df["TimeGroup"] = copy_df["TimeGroup"].astype(str)
    main_results=[]
    for i, (before, after) in enumerate(zip(before_ramp, after_ramp)):
        q_before = (copy_df[copy_df["New_Measurement"] == before]
                    .groupby("TimeGroup")
                    .size()
                    .reset_index(name="q_before"))

        q_after = (copy_df[copy_df["New_Measurement"] == after]
                   .groupby("TimeGroup")
                   .size()
                   .reset_index(name="q_after"))

        merged = q_before.merge(q_after, on="TimeGroup", how="outer").fillna(0)
        merged["Qm"] = (merged["q_before"] + merged["q_after"]) / 2
        main_results.append(merged)

    main_df = pd.concat(main_results, ignore_index=True)
    main_df = main_df.groupby("TimeGroup")[["Qm"]].mean().reset_index()

    ramp_results = []
    for input_, output_ in zip(input_ramp, output_ramp):
        q_in = (copy_df[copy_df["New_Measurement"] == input_]
                .groupby("TimeGroup").size().reset_index(name="q_in"))

        q_out = (copy_df[copy_df["New_Measurement"] == output_]
                 .groupby("TimeGroup").size().reset_index(name="q_out"))
        q_out["q_out"] = 0
        merged = q_in.merge(q_out, on="TimeGroup", how="outer").fillna(0)
        ramp_results.append(merged)

    ramp_df = pd.concat(ramp_results, ignore_index=True)
    ramp_df = ramp_df.groupby("TimeGroup")[["q_in", "q_out"]].mean().reset_index()

    rfr_df = pd.merge(main_df, ramp_df, on="TimeGroup", how="inner")

    # 램프 간섭 영향률 계산
    rfr_df["IR_in"] = rfr_df["q_in"] / rfr_df["Qm"]
    rfr_df["IR_out"] = rfr_df["q_out"] / rfr_df["Qm"]
    rfr_df["RFR"] = rfr_df["IR_in"] + rfr_df["IR_out"]

    # -----------------------------
    # 특정 검지기에만 RFR 반영
    # -----------------------------
    target_measurements = after_ramp   # RFR 적용 대상 검지기 번호
    all_measurements = copy_df["New_Measurement"].unique()

    expanded_df_list = []
    for m in all_measurements:
        temp = rfr_df.copy()
        temp["New_Measurement"] = m
        # 지정된 구간에만 RFR 값 유지, 나머지는 0
        temp["RFR"] = temp["RFR"] if m in target_measurements else 0
        expanded_df_list.append(temp)

    final_rfr_df = pd.concat(expanded_df_list, ignore_index=True)
    final_rfr_df = final_rfr_df.sort_values(by=["TimeGroup", "New_Measurement"]).reset_index(drop=True)
    final_rfr_df["RFR"] = 0
    return final_rfr_df

# 진출 원활율
def output_normality(original_df):
    copy_df = original_df.copy()
    entry_df = copy_df[copy_df["New_Measurement"].isin(enter_line)][["VehNo", "t(Entry)"]]

    exit_df = copy_df[copy_df["New_Measurement"].isin(before_ramp)][["VehNo", "t(Entry)"]]

    # 차량 번호로 그룹화 후 시간의 최솟값(중복제거)
    entry_first = (
        entry_df.groupby("VehNo")["t(Entry)"].min()
        .reset_index()  # Series → DataFrame
        .rename(columns={"t(Entry)": "t_entry"})
    )

    exit_first = (
        exit_df.groupby("VehNo")["t(Entry)"].min()
        .reset_index()
        .rename(columns={"t(Entry)": "t_exit"})
    )

    # 지연시간
    merged = pd.merge(entry_first, exit_first, on="VehNo", how="inner")
    merged["delay_sec"] = merged["t_exit"] - merged["t_entry"]
    merged = merged[merged["delay_sec"] >= 0]  # 음수 제거

    # 지연시간(중앙값) → lag_bins
    if len(merged) and np.isfinite(np.nanmedian(merged["delay_sec"])): # delay_sec의 값이 유효하면
        lag_bins = int(round(np.nanmedian(merged["delay_sec"]) / 300)) # 단위시간으로 나눴을 때의 중간값 => 3(900초) => 진입한 차량이 진출을 통과하는데 평균 900초가 걸림
    else:
        lag_bins = 0  # 데이터 부족 시 동시간 매칭

    # TimeGroup별 진입/유출 카운트 집계
    entry_count = (original_df[original_df["New_Measurement"].isin(enter_line)]
                .groupby("TimeGroup").size().reset_index(name="Q_in"))
    exit_count  = (original_df[original_df["New_Measurement"].isin(before_ramp)]
                .groupby("TimeGroup").size().reset_index(name="Q_out"))

    merged_counts = pd.merge(entry_count, exit_count, on="TimeGroup", how="left")

    # Q_out을 지연 시간만큼 shift
    merged_counts["Q_out_shift"] = merged_counts["Q_out"].shift(-lag_bins)

    # F(outrate) 계산
    merged_counts["F(outrate)"] = (merged_counts["Q_out_shift"] / merged_counts["Q_in"]).fillna(0)

    all_measurements = copy_df["New_Measurement"].unique()
    expanded_list = []

    for m in all_measurements:
        temp = merged_counts.copy()
        temp["New_Measurement"] = m
        if m not in before_ramp:  # 222 이외의 검지기
            temp["F(outrate)"] = 0
        expanded_list.append(temp)

    final_df = pd.concat(expanded_list, ignore_index=True)
    final_df = final_df.sort_values(by=["TimeGroup", "New_Measurement"]).reset_index(drop=True)
    final_df["F(outrate)"] = 0
    return final_df


def calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df):

    # TimeGroup 기준으로  Merge
    merged_df = (
        speed_df[["TimeGroup", "New_Measurement", "delta_V"]]
        .merge(density_df[["TimeGroup", "New_Measurement", "delta_K"]], on=["TimeGroup", "New_Measurement"])
        .merge(heavy_df[["TimeGroup", "New_Measurement", "rate"]], on=["TimeGroup", "New_Measurement"])
        .merge(entry_saturation_df[["TimeGroup", "New_Measurement", "Phi_진입"]], on=["TimeGroup", "New_Measurement"])
        .merge(rfr_df[["TimeGroup", "New_Measurement", "RFR"]], on=["TimeGroup", "New_Measurement"])
        .merge(normality_df[["TimeGroup", "New_Measurement", "F(outrate)"]], on=["TimeGroup", "New_Measurement"])
    )

    merged_df["STVM"] = (
        weights["w1"] * merged_df["delta_V"] +
        weights["w2"] * merged_df["delta_K"] +
        weights["w3"] * merged_df["rate"] +
        weights["w4"] * merged_df["Phi_진입"] +
        weights["w5"] * merged_df["RFR"] +
        weights["w6"] * merged_df["F(outrate)"]
    )
    merged_df = modify_frame(merged_df)
    return merged_df


def calculate_z_score(stvm_df):
    copy_df = stvm_df.copy()

    # 평균
    mean_stvm = copy_df["STVM"].mean(skipna=True)

    # 표준편차
    std_stvm = copy_df["STVM"].std(skipna=True)

    # Z-Score 계산
    copy_df["Z-Score"] = (copy_df["STVM"] - mean_stvm) / std_stvm
    z_max = copy_df["Z-Score"].max()
    z_min = copy_df["Z-Score"].min()

    copy_df["환산점수"] = copy_df["Z-Score"].apply(lambda z : z_to_score(z, z_min, z_max))

    stvm_df = pd.pivot(copy_df, index="TimeGroup", columns= "New_Measurement", values="환산점수")
    return stvm_df

def modify_frame(df):
    modify_df = df.copy()

    modify_df["StartTime"] = modify_df["TimeGroup"].str.split("~").str[0].astype(int)
    modify_df = modify_df[(modify_df["StartTime"] >=1800) &(modify_df["StartTime"] < 5400)]
    modify_df = modify_df[~modify_df["New_Measurement"].isin([280,999])]
    return modify_df


def variable_timegroup_avg(stvm_df):
    copy_df = stvm_df.copy()
    variable_time_df = copy_df.groupby("TimeGroup")[["delta_V", "delta_K", "rate", "Phi_진입", "RFR", "F(outrate)"]].mean()
    return variable_time_df

def variable_total_avg(variable_df):
    variable_total_df = pd.DataFrame([variable_df.mean(numeric_only=True)])
    return variable_total_df

def speed_density_avg(density_df):
    copy_df = density_df.copy()
    avg_df = modify_frame(copy_df)
    avg_df = pd.DataFrame([avg_df.mean(numeric_only=True)])
    avg_df = avg_df[["V_mean", "K"]]
    return avg_df

def pivot_table(df, value, preprocess=None):
    copy_df = df.copy()
    if preprocess :
        copy_df = preprocess(copy_df)
    return copy_df.pivot(index="TimeGroup", columns="New_Measurement", values=value)

def weighted_avg_speed(original_df):
    copy_df = original_df.copy()
    # TimeGroup, New_Measurement별 그룹화 및 속도 평균
    speed_mean_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement", "Vehicle type"])
          .agg(V_mean=("v[km/h]", "mean"), V_count=("v[km/h]", "count"))
          .reset_index()
    )
    speed_mean_df["std_group"] = speed_mean_df.groupby(["TimeGroup", "New_Measurement"])["V_mean"].transform(lambda s: s.std(ddof=0))
    speed_mean_df["cv"] = speed_mean_df["std_group"] / speed_mean_df["V_mean"]
    speed_mean_df["w"] = 1 / speed_mean_df["cv"]
    speed_mean_df["w*v"] = speed_mean_df["w"] * speed_mean_df["V_mean"]

    weighted_result = (
        speed_mean_df.groupby(["TimeGroup","New_Measurement"])
          .apply(lambda g: g["w*v"].sum() / g["w"].sum())
          .reset_index(name="Weighted_Avg_Speed")
    )

    return weighted_result

def save_to_excel(excel_df, folder_path, file_name, i):
        excel_folder_path = os.path.join(folder_path, file_name)
        os.makedirs(excel_folder_path, exist_ok=True)
        excel_file_name = f"{file_name}_{i+1}.xlsx"
        excel_file_path = os.path.join(excel_folder_path, excel_file_name)
        excel_df.to_excel(excel_file_path, index=True)
        print(f"{excel_file_name} 생성 완료")

def z_to_score(z, z_min, z_max):
    if 1.645 <= z <= z_max:
        return 50 + ((95 + 5 * ((z - 1.645) / (z_max - 1.645))) * 0.5)
    elif 1.282 <= z < 1.645:
        return 50 + ((90 + 5 * ((z - 1.282) / (1.645 - 1.282))) * 0.5)
    elif 1.038 <= z < 1.282:
        return 50 + ((85 + 5 * ((z - 1.038) / (1.282 - 1.038))) * 0.5)
    elif 0.842 <= z < 1.038:
        return 50 + ((80 + 5 * ((z - 0.842) / (1.038 - 0.842))) * 0.5)
    elif 0.676 <= z < 0.842:
        return 50 + ((75 + 5 * ((z - 0.676) / (0.842 - 0.676))) * 0.5)
    elif 0.526 <= z < 0.676:
        return 50 + ((70 + 5 * ((z - 0.526) / (0.676 - 0.526))) * 0.5)
    elif 0.387 <= z < 0.526:
        return 50 + ((65 + 5 * ((z - 0.387) / (0.526 - 0.387))) * 0.5)
    elif 0.255 <= z < 0.387:
        return 50 + ((60 + 5 * ((z - 0.255) / (0.387 - 0.255))) * 0.5)
    elif -0.255 <= z < 0.255:
        return 50 + ((40 + 5 * ((z + 0.255) / (0.255 + 0.255))) * 0.5)
    elif -0.387 <= z < -0.255:
        return 50 + ((35 + 5 * ((z + 0.387) / (-0.255 + 0.387))) * 0.5)
    elif -0.526 <= z < -0.387:
        return 50 + ((30 + 5 * ((z + 0.526) / (-0.387 + 0.526))) * 0.5)
    elif -0.676 <= z < -0.526:
        return 50 + ((25 + 5 * ((z + 0.676) / (-0.676 + 0.842))) * 0.5)
    elif -0.842 <= z < -0.676:
        return 50 + ((20 + 5 * ((z + 0.842) / (-0.676 + 0.842))) * 0.5)
    elif -1.038 <= z < -0.842:
        return 50 + ((15 + 5 * ((z + 1.038) / (-0.842 + 1.038))) * 0.5)
    elif -1.282 <= z < -1.038:
        return 50 + ((10 + 5 * ((z + 1.282) / (-1.038 + 1.282))) * 0.5)
    elif -1.645 <= z < -1.282:
        return 50 + ((5 + 5 * ((z + 1.645) / (-1.282 + 1.645))) * 0.5)
    elif z_min <= z < -1.645:
        return 50 + ((0 + 5 * ((z + z_min) / (-1.645 + z_min))) * 0.5)
    else:
        return np.nan
###################################################################################################################

folder_path = path
mer_list = sorted(
    [file for file in os.listdir(folder_path) if file.endswith(".mer")],
    key=lambda x: int(re.search(r"(\d+)(?=\.mer$)", x).group()) if re.search(r"(\d+)(?=\.mer$)", x) else 0
)

grouped_df = pd.DataFrame()
result_df = pd.DataFrame()


for i in range(len(mer_list)):
    mer_file = mer_list[i]
    print("작업파일 : ", mer_file)
    with open(os.path.join(folder_path, mer_file), "r", encoding="utf-8", errors="ignore") as file:
            lines = file.readlines()
            # 데이터가 시작하는 인덱스 찾기
            data_start_idx = None

            for j, line in enumerate(lines):
                if "Measurem." in line:  # 컬럼명이 포함된 행 찾기
                    data_start_idx = j
                    break

            # 데이터프레임 생성
            if data_start_idx is not None:

                # 컬럼명 추출 및 공백 제거
                columns = [col.strip() for col in lines[data_start_idx].strip().split(";")]

                # 데이터 부분 추출 및 가공
                data_lines = lines[data_start_idx + 1:]  # 컬럼명 제외, 데이터 부분
                data = [line.strip().split(";") for line in data_lines if line.strip()]

                # 데이터프레임 생성
                df = pd.DataFrame(data, columns=columns)

                # 컬럼 내부 데이터 정수형 변환
                df = df.apply(pd.to_numeric, errors="coerce")

                original_df = df[(df["t(Entry)"] != -1.00)].reset_index(drop=True)


                #불필요 컬럼 제거
                original_df.drop(columns=["b[m/s2]", "tQueue", "Occ", "Pers"], inplace=True, errors="ignore")

                original_df["New_Measurement"] = original_df["Measurem."] % 1000

                bins = np.arange(start_interval, end_interval+1, 300)
                labels = [f"{start}~{start+300}" for start in bins[:-1]]  # 구간 라벨링

                # 구간 나누기 및 컬럼 추가
                original_df["TimeGroup"] = pd.cut(original_df["t(Entry)"], bins=bins, labels=labels, right=False)

                # 속도변화율
                speed_df = speed_mean(original_df)

                # 밀도변화율
                density_df = density_mean(speed_df)

                # 중차량 혼입률
                heavy_df = heavy_rate(original_df)

                # 동적 포화도
                entry_saturation_df = entry_saturation(original_df)

                # 램프 간섭 영향률
                rfr_df = rfr_rate(original_df)

                # 진출 원활율
                normality_df = output_normality(original_df)

                # STVM 계산
                stvm_df = calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df)
                #display("stvm_df : ", stvm_df)

                # Z-Score 계산
                z_score_df = calculate_z_score(stvm_df)
                display("z_score_df : ", z_score_df)
                save_to_excel(z_score_df, folder_path, "환산점수", i)

                # STVM 피봇
                #stvm_pivot_df = pivot_table(stvm_df, "STVM")
                #display("stvm_pivot_df : ", stvm_pivot_df)
                #save_to_excel(stvm_pivot_df, folder_path, "STVM", i)

                # 속도값 피봇
                #speed_pivot_df = pivot_table(speed_df, "V_mean", preprocess=modify_frame)
                #display("speed_pivot_df : ", speed_pivot_df)
                #save_to_excel(speed_pivot_df, folder_path, "속도값", i)

                # 메모리 정리
                #del df, original_df, speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df, stvm_df, z_score_df
                gc.collect()


작업파일 :  ToyNetwork_진입램프_600.mer


'z_score_df : '

New_Measurement,1,2,3,4,5,6,7,8,9,10,...,270,271,272,273,274,275,276,277,278,279
TimeGroup,,,,,,,,,,,,,,,,,,,,,
1800~2100,52.660201,52.809800,52.712389,52.760061,52.626720,52.654645,52.691606,52.597515,52.670682,52.670647,...,71.974121,72.080372,70.546170,71.206322,71.203989,72.041169,72.457497,71.954407,83.485802,82.854703
2100~2400,52.824293,52.626726,52.804388,52.721275,52.721287,52.721273,52.618410,52.855277,52.937832,52.766686,...,83.025478,83.746844,85.446071,86.128834,86.814208,84.783476,84.995068,85.558628,84.243636,72.088327
2400~2700,52.706217,52.695442,52.638858,52.797600,52.866505,52.763434,52.717052,52.706341,52.557371,52.562339,...,72.397931,71.922946,72.434050,72.428672,72.151346,71.772151,71.840724,72.023289,71.738853,82.157539
2700~3000,52.786865,52.798432,52.809878,52.727758,52.763442,52.809848,52.821251,52.738569,52.821296,52.832664,...,86.390416,90.336537,86.626761,86.271602,82.224920,80.064553,71.801100,72.332915,82.501065,80.617137
3000~3300,56.937111,58.528549,59.734096,58.252735,60.222088,57.772547,56.750515,57.131394,57.779785,57.778047,...,71.874586,71.665853,72.021070,71.736481,84.289003,86.267668,88.702764,86.248424,82.435704,88.434497
3300~3600,52.782378,52.730500,52.978672,52.810806,52.906761,52.757545,52.739365,52.772841,52.814341,52.814294,...,81.830469,82.648135,84.399863,84.621048,82.601878,82.178961,72.183134,83.037063,84.227306,72.426599
3600~3900,57.180139,54.029900,57.368899,54.718333,55.029402,54.893026,54.892895,52.636515,53.817864,52.662728,...,85.182395,83.830469,82.059316,72.182735,80.628880,84.193733,86.234793,84.079526,83.679469,84.282694
3900~4200,52.662733,52.706541,52.762201,52.741064,52.696440,52.774029,52.752631,52.537953,52.761732,52.617877,...,71.400842,70.144598,70.564979,70.567122,67.711618,68.292015,67.450883,67.444762,67.444476,66.413962
4200~4500,52.990078,52.896485,52.834160,52.814253,52.804332,52.844607,52.995558,52.875853,52.772484,52.804013,...,85.191339,90.274932,91.977762,93.442584,93.972676,92.758898,92.762926,89.666901,89.567911,92.367847


환산점수_1.xlsx 생성 완료
작업파일 :  ToyNetwork_진입램프_1000.mer


'z_score_df : '

...

New_Measurement,1,2,3,4,5,6,7,8,9,10,...,270,271,272,273,274,275,276,277,278,279
TimeGroup,,,,,,,,,,,,,,,,,,,,,
1800~2100,52.588540,52.593037,52.583082,52.585909,52.600903,52.611738,52.602293,52.585116,52.590909,52.630717,...,80.483759,80.634766,80.062509,72.109489,71.884561,72.445942,71.975193,80.756790,82.050507,85.820838
2100~2400,56.995256,55.606894,54.144420,53.324251,53.782467,53.056419,53.402224,52.504694,52.591573,54.398883,...,97.551151,97.533163,97.345685,97.036912,97.504134,96.866067,96.898445,96.196907,95.003111,93.363588
2400~2700,54.805538,55.081673,55.554164,56.453160,56.260659,57.247099,56.743790,56.822844,55.441586,54.646650,...,84.796995,85.898656,86.635724,86.510310,84.752167,88.082075,88.894340,90.303902,91.514654,90.833208
2700~3000,52.595462,52.572305,52.565011,52.593168,52.580472,52.607430,52.606345,52.584137,52.565079,52.568013,...,87.359539,85.258351,86.698079,89.576091,89.616104,86.784492,83.390188,80.182825,72.496306,80.692493
3000~3300,56.107153,55.037721,54.558315,55.210586,55.693911,57.132025,55.269436,55.296259,55.123505,54.871526,...,85.995039,88.994386,87.912175,82.950838,82.938972,83.982577,88.289427,87.870326,87.460877,89.967501
3300~3600,52.634298,52.608510,52.585918,52.590148,52.606727,52.602309,52.578534,52.587308,52.602291,52.596663,...,69.748366,70.238640,70.597842,71.783567,72.144316,72.330161,72.031705,72.350563,72.444447,72.093752
3600~3900,55.764261,54.109526,52.886335,54.181942,53.493328,52.509225,52.513876,52.539036,52.537107,52.545543,...,91.526405,90.002651,87.233929,84.176274,84.111374,85.087808,82.051291,81.948142,83.042948,86.401803
3900~4200,57.456381,57.908190,58.298638,56.828936,57.773010,59.570332,59.134835,60.790907,60.882954,60.718500,...,72.201179,71.821881,80.555013,82.391297,72.399013,71.546209,71.378897,71.272624,70.592287,67.648537
4200~4500,54.606869,54.559007,55.546596,56.224516,54.846526,54.761270,55.652491,55.147041,54.972386,54.381782,...,69.048227,70.454353,70.914661,71.113464,72.172388,82.333181,86.153056,89.102804,90.035705,92.038007


환산점수_4.xlsx 생성 완료
작업파일 :  ToyNetwork_진입램프_2200.mer


'z_score_df : '

New_Measurement,1,2,3,4,5,6,7,8,9,10,...,270,271,272,273,274,275,276,277,278,279
TimeGroup,,,,,,,,,,,,,,,,,,,,,
1800~2100,68.225694,66.525003,63.133062,56.457982,57.688217,55.634779,57.379684,55.080193,54.892413,55.508382,...,82.915148,72.427206,71.966032,71.367161,71.339731,71.479578,71.720259,80.066524,84.237659,83.253459
2100~2400,54.454814,54.111845,54.339212,59.539440,54.767802,54.702452,56.201739,60.953397,57.280390,55.590214,...,72.184999,81.102540,72.429184,80.304239,72.406454,72.406218,80.768060,81.236569,71.574246,71.639658
2400~2700,60.011821,57.665173,52.515551,52.512957,55.373736,54.291621,52.507877,52.523769,53.516840,52.503781,...,96.077129,95.874996,96.013873,95.652471,94.950666,93.709036,92.090854,90.848110,93.272535,93.611964
2700~3000,52.504250,52.528636,53.564211,53.387035,52.506236,52.501280,52.500985,53.425755,53.568945,53.116206,...,80.259289,72.303346,80.385273,72.165963,81.376837,81.632848,80.089478,72.314079,71.389909,71.784676
3000~3300,52.516746,52.520837,52.506280,52.520126,52.500663,52.525832,52.512388,52.503302,52.521372,52.510255,...,71.831861,71.935009,71.915041,72.464527,71.862655,71.973334,72.392508,72.442585,83.749752,86.172296
3300~3600,57.920184,54.701225,56.077554,56.552285,54.521811,54.810815,55.424064,54.303067,52.502051,54.429573,...,71.980575,72.127565,71.942083,71.636426,71.812142,71.621889,71.349584,71.556831,70.278347,68.213718
3600~3900,52.515546,52.535975,52.526021,52.514222,52.504196,52.542202,52.537691,52.544839,52.721472,52.529296,...,80.024543,81.919029,82.180473,85.669307,88.362596,90.796408,90.918733,91.224876,92.846728,92.259790
3900~4200,60.295965,66.084530,58.797394,53.720272,54.109724,58.149006,56.875297,59.124286,56.068304,55.665356,...,84.086911,83.782794,82.324121,72.314209,71.431212,70.698877,70.536425,70.008125,68.150282,69.208352
4200~4500,56.504889,52.678313,55.918906,56.770886,54.858182,55.450881,54.327725,54.547674,55.524652,57.256592,...,95.088843,94.593141,94.866869,95.253924,95.066784,94.859209,95.092254,95.616357,95.583365,95.327044


환산점수_5.xlsx 생성 완료
